In [ ]:
# Cell 1: Imports and Setup
from typing import Dict, List, Any, Optional
import tempfile
import os
import shutil
from collections import defaultdict
import json

from module.agent_document_summarize import process_documents
from module.agent_human_management import analyze_team_roles, validate_team_details
from module.graph_updates import Neo4jCRUD
from module.get_model_and_embeding import get_llm

# Neo4j configuration
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "neo4j123"

# Cell 2: Helper Functions
def save_files_to_temp(file_paths: List[str]) -> tuple[List[str], str]:
    """Save files to temporary directory"""
    temp_dir = tempfile.mkdtemp()
    temp_paths = []
    
    for file_path in file_paths:
        if not file_path.lower().endswith(('.pdf', '.txt', '.md')):
            raise ValueError(f"Unsupported file type: {file_path}")
        
        file_name = os.path.basename(file_path)
        temp_path = os.path.join(temp_dir, file_name)
        shutil.copy2(file_path, temp_path)
        temp_paths.append(temp_path)
    
    return temp_paths, temp_dir

def distribute_tasks(db: Neo4jCRUD, workspace_id: str) -> Dict[str, List[Dict[str, Any]]]:
    """Distribute tasks evenly among team members"""
    with db.driver.session(database=workspace_id) as session:
        # Get all tasks and their associated roles
        result = session.run("""
            MATCH (w:Workspace {name: $workspace})-[:CONTAINS_ROLE]->(r:Role)-[:HAS_TASK]->(t:Task)
            MATCH (p:Person)-[:CAN_PERFORM]->(r)
            RETURN p.name as person, r.name as role, t.name as task, ID(t) as task_id
        """, workspace=workspace_id)
        
        # Organize tasks by person
        tasks_by_person = defaultdict(list)
        for record in result:
            person = record["person"]
            tasks_by_person[person].append({
                "task": record["task"],
                "role": record["role"],
                "node_id": record["task_id"]
            })
            
        return dict(tasks_by_person)

# Cell 3: Neo4j Graph Creation Function
def create_neo4j_graph(
    workspace_id: str,
    document_analysis: Dict[str, List[str]],
    team_analysis: Dict[str, List[str]]
) -> None:
    """Create Neo4j graph with relationships"""
    db = Neo4jCRUD(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD, workspace_id)
    
    try:
        # Create workspace node
        db.create_node("Workspace", {"name": workspace_id})
        print(f"Created workspace node: {workspace_id}")
        
        # Process roles and tasks from document analysis
        for role, tasks in document_analysis.items():
            if role != "_processing_summary":
                # Create role node
                db.create_node("Role", {
                    "name": role,
                    "type": "role"
                })
                print(f"Created role node: {role}")
                
                # Link role to workspace
                db.create_relationship(
                    "Workspace", workspace_id,
                    "Role", role,
                    "CONTAINS_ROLE"
                )
                
                # Create task nodes and link to role
                for task in tasks:
                    db.create_node("Task", {
                        "name": task,
                        "type": "task"
                    })
                    db.create_relationship(
                        "Role", role,
                        "Task", task,
                        "HAS_TASK"
                    )
                print(f"Created {len(tasks)} tasks for role: {role}")
        
        # Process team members and their possible roles
        for member_name, possible_roles in team_analysis.items():
            # Create team member node
            db.create_node("Person", {
                "name": member_name,
                "type": "person"
            })
            print(f"Created person node: {member_name}")
            
            # Link member to workspace
            db.create_relationship(
                "Workspace", workspace_id,
                "Person", member_name,
                "HAS_MEMBER"
            )
            
            # Link member to possible roles
            for role in possible_roles:
                if role in document_analysis:
                    db.create_relationship(
                        "Person", member_name,
                        "Role", role,
                        "CAN_PERFORM"
                    )
            print(f"Created {len(possible_roles)} role relationships for: {member_name}")
    
    finally:
        db.close()

# Cell 4: Main Processing Function
def process_and_store(
    workspace_id: str,
    file_paths: List[str],
    team_details: Dict[str, Dict[str, Any]]
) -> Dict[str, Any]:
    """Process documents and team analysis, storing results in Neo4j"""
    try:
        # Validate team details
        validate_team_details(team_details)
        print("Team details validated")
        
        # Save files to temp directory
        temp_paths, temp_dir = save_files_to_temp(file_paths)
        print(f"Files saved to temporary directory: {temp_dir}")
        
        try:
            # Initialize LLM
            llm = get_llm()
            print("LLM initialized")
            
            # Process documents
            document_analysis = process_documents(
                document_paths=temp_paths,
                current_roles=[details["current_role"] for details in team_details.values()]
            )
            print("Document analysis completed")
            
            # Analyze team roles
            team_analysis = analyze_team_roles(team_details, llm)
            print("Team role analysis completed")
            
            # Create Neo4j graph
            create_neo4j_graph(workspace_id, document_analysis, team_analysis)
            print("Neo4j graph created")
            
            return {
                "status": "success",
                "document_analysis": document_analysis,
                "team_analysis": team_analysis
            }
            
        finally:
            shutil.rmtree(temp_dir)
            print("Temporary directory cleaned up")
            
    except Exception as e:
        print(f"Error in processing: {str(e)}")
        raise

# Cell 5: Graph Management Functions
def delete_node(workspace_id: str, node_id: str, node_name: str) -> Dict[str, str]:
    """Delete a node from the graph"""
    db = Neo4jCRUD(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD, workspace_id)
    try:
        with db.driver.session(database=workspace_id) as session:
            session.run("""
                MATCH (n)
                WHERE ID(n) = $node_id AND n.name = $node_name
                DETACH DELETE n
            """, node_id=int(node_id), node_name=node_name)
            
        return {"status": "success", "message": f"Node {node_name} deleted"}
    finally:
        db.close()

def update_node(
    workspace_id: str,
    node_id: str,
    node_name: str,
    new_properties: Dict[str, Any]
) -> Dict[str, str]:
    """Update a node's properties"""
    db = Neo4jCRUD(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD, workspace_id)
    try:
        with db.driver.session(database=workspace_id) as session:
            session.run("""
                MATCH (n)
                WHERE ID(n) = $node_id AND n.name = $node_name
                SET n += $properties
            """, 
                node_id=int(node_id),
                node_name=node_name,
                properties=new_properties
            )
        return {"status": "success", "message": f"Node {node_name} updated"}
    finally:
        db.close()

def get_workspace_tasks(workspace_id: str) -> Dict[str, List[Dict[str, Any]]]:
    """Get distributed tasks for a workspace"""
    db = Neo4jCRUD(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD, workspace_id)
    try:
        return distribute_tasks(db, workspace_id)
    finally:
        db.close()

In [15]:
# Test data
test_workspace = "wakuwakuanya"

# Sample team details
test_team = {
    "John Doe": {
        "current_role": "Software Developer",
        "skills": ["Python", "JavaScript", "Docker"],
        "experience": "5 years in web development"
    },
    "Jane Smith": {
        "current_role": "UX Designer",
        "skills": ["UI/UX Design", "Figma", "User Research"],
        "experience": "3 years in product design"
    }
}

# Sample document paths
test_docs = [
    r"D:\Mindforge\AIService\test_doc\doc1.pdf"
]

In [16]:
# Process documents and create graph
print("Starting processing...")
result = process_and_store(test_workspace, test_docs, test_team)
print("\nProcessing result:", json.dumps(result, indent=2))

Starting processing...
Team details validated
Files saved to temporary directory: C:\Users\Tanap\AppData\Local\Temp\tmpzjoll9wp
LLM initialized
Loading documents from 1 paths...
Loading documents...
Loaded 27 documents.
Loaded 27 documents successfully.

Processing document 1/27...
Successfully processed document 1

Processing document 2/27...
Successfully processed document 2

Processing document 3/27...
Successfully processed document 3

Processing document 4/27...
Successfully processed document 4

Processing document 5/27...
Successfully processed document 5

Processing document 6/27...
Successfully processed document 6

Processing document 7/27...
Successfully processed document 7

Processing document 8/27...
Successfully processed document 8

Processing document 9/27...
Successfully processed document 9

Processing document 10/27...
Successfully processed document 10

Processing document 11/27...
Successfully processed document 11

Processing document 12/27...
Successfully proces

In [17]:

# Get distributed tasks
print("\nRetrieving tasks...")
tasks = get_workspace_tasks(test_workspace)
print("Task distribution:", json.dumps(tasks, indent=2))


Retrieving tasks...


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 4, column: 70, offset: 219} for query: '\n            MATCH (w:Workspace {name: $workspace})-[:CONTAINS_ROLE]->(r:Role)-[:HAS_TASK]->(t:Task)\n            MATCH (p:Person)-[:CAN_PERFORM]->(r)\n            RETURN p.name as person, r.name as role, t.name as task, ID(t) as task_id\n        '


Task distribution: {
  "John Doe": [
    {
      "task": "Identify events and analyze the applicability of actions",
      "role": "Software Developer",
      "node_id": 51
    },
    {
      "task": "Construct dynamic model diagram, comprising of state transition diagrams",
      "role": "Software Developer",
      "node_id": 134
    },
    {
      "task": "Determine the level of coupling among modules",
      "role": "Software Developer",
      "node_id": 133
    },
    {
      "task": "Identify classes",
      "role": "Software Developer",
      "node_id": 132
    },
    {
      "task": "Determine the degree of intra-dependability within elements of a module",
      "role": "Software Developer",
      "node_id": 131
    },
    {
      "task": "Generate a Requirements Speci\ufb01cation Document",
      "role": "Software Developer",
      "node_id": 130
    },
    {
      "task": "Define user object attributes",
      "role": "Software Developer",
      "node_id": 129
    },
    {
   

In [20]:

# Test node update
print("\nUpdating node...")
update_result = update_node(
    test_workspace,
    "1",  # example node ID
    "Some Task",  # example node name
    {"priority": "high"}
)
print("Update result:", update_result)


Updating node...


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 39, offset: 49} for query: '\n                MATCH (n)\n                WHERE ID(n) = $node_id AND n.name = $node_name\n                SET n += $properties\n            '


Update result: {'status': 'success', 'message': 'Node Some Task updated'}


In [19]:
    
# Test node deletion
print("\nDeleting node...")
delete_result = delete_node(
    test_workspace,
    "1",  # example node ID
    "Some Task"  # example node name
)
print("Delete result:", delete_result)


Deleting node...


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 3, column: 39, offset: 49} for query: '\n                MATCH (n)\n                WHERE ID(n) = $node_id AND n.name = $node_name\n                DETACH DELETE n\n            '


Delete result: {'status': 'success', 'message': 'Node Some Task deleted'}
